# UAS Risk Analysis Visualization

## Setup

In [1]:
!python -m pip install -q geopandas seaborn s3fs pyarrow fiona

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-scheduler 2.9.0 requires fsspec==2023.6.0, but you have fsspec 2024.10.0 which is incompatible.


### Imports

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
from glob import glob
import subprocess

### Configurations

In [3]:
sns.set_style('darkgrid')

## Load UAS Risk Analysis Results

In [4]:
risk_scores_base_path = 's3://endurasoft-dev-risk-framework/analysis/uas_risk_scores/v3/'

In [5]:
risk_scores_df = pd.read_csv(os.path.join(risk_scores_base_path, 'consolidated/uas_risk_scores_csv/uas_risk_scores.csv'), low_memory=False)
risk_scores_df.head()

,globalid,objectid,ceiling,unit,map_eff,last_edit,latitude,longitude,arpt_count,apt1_faaid,...,shape__length,shape__area,shape__area_2,shape__length_2,wkt,flight_int_count,uas_sighting_int_count,risk_factor,risk_score,risk_label
0,00017c46-d5bc-4c72-b379-127c8c63de2d,260889,0,Feet,11/28/2024,9/12/2019,21.979170,-159.804170,1,BKH,...,NaN,NaN,0.000069,0.033333,POLYGON Z ((-159.808333318407 21.9750038051595...,0,0,0.0,0.0,LOW
1,000280b8-1558-4226-95cb-f53df70d6308,19192,150,Feet,11/28/2024,8/30/2018,30.420837,-84.304180,1,TLH,...,NaN,NaN,0.000069,0.033333,POLYGON Z ((-84.3083413416534 30.4166713993425...,0,0,0.0,0.0,LOW
2,0002afa2-18e6-4d01-80f5-6004198792da,367316,400,Feet,11/28/2024,4/27/2017,70.137505,-148.670840,1,SCC,...,NaN,NaN,0.000069,0.033333,POLYGON Z ((-148.67499659182 70.1333363769282 ...,0,0,0.0,0.0,LOW
3,0002e98b-e4a9-450d-bd36-880fb8ea068d,142265,400,Feet,11/28/2024,12/7/2017,40.054173,-76.329170,1,LNS,...,NaN,NaN,0.000069,0.033333,POLYGON Z ((-76.3333397470069 40.0500066587629...,0,0,0.0,0.0,LOW
4,00033355-eac9-45ac-b117-809f80a9de84,133052,200,Feet,11/28/2024,10/11/2017,39.495840,-74.520836,1,ACY,...,NaN,NaN,0.000069,0.033333,POLYGON Z ((-74.5250060519168 39.4916732139747...,0,0,0.0,0.0,LOW


In [6]:
risk_scores_df.shape

(376569, 51)

In [8]:
# Rename long target columns (columns with more than 10 characters are truncated)
risk_scores_df.rename({'flight_int_count': 'flight_int', 'uas_sighting_int_count': 'uas_int'}, axis=1, inplace=True)

## Convert to GeoPandas and Save Shapefile

In [9]:
# Convert risk scores dataframe to geopandas
risk_scores_gdf = gpd.GeoDataFrame(
    risk_scores_df,
    geometry=gpd.GeoSeries.from_wkt(risk_scores_df['wkt'])
)

In [10]:
shapefile_dir = 'uas_risk_scores_shapefile'
os.makedirs(shapefile_dir, exist_ok=True)
risk_scores_gdf.to_file(os.path.join(shapefile_dir, 'uas_risk_scores.shp'), driver='ESRI Shapefile', engine='fiona', crs='EPSG:4326')

/tmp/ipykernel_121/343700671.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  risk_scores_gdf.to_file(os.path.join(shapefile_dir, 'uas_risk_scores.shp'), driver='ESRI Shapefile', engine='fiona', crs='EPSG:4326')


In [11]:
glob(os.path.join(shapefile_dir, '*.*'))

['uas_risk_scores_shapefile/uas_risk_scores.shx',
 'uas_risk_scores_shapefile/uas_risk_scores.shp',
 'uas_risk_scores_shapefile/uas_risk_scores.prj',
 'uas_risk_scores_shapefile/uas_risk_scores.dbf',
 'uas_risk_scores_shapefile/uas_risk_scores.cpg']

In [12]:
# Create a zip file
with zipfile.ZipFile('uas_risk_scores_shapefile.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add all shapefile components to zip
    for path in glob(os.path.join(shapefile_dir, '*.*')):
        zipf.write(path)

In [13]:
# Upload to s3
shapefile_outpath = os.path.join(risk_scores_base_path, 'viz/uas_risk_scores_shapefile.zip')
subprocess.check_call(f'aws s3 cp uas_risk_scores_shapefile.zip {shapefile_outpath}'.split())

upload: ./uas_risk_scores_shapefile.zip to s3://endurasoft-dev-risk-framework/analysis/uas_risk_scores/v3/viz/uas_risk_scores_shapefile.zip


0